先使用暴力数数，快速建立一个验证函数。数的关键是，不重复/不遗漏。
可以把所有三角形的形态，分为8种：上下左右4个方向，每个方向两个（附着在水平或垂直线上）。

In [26]:

def tri(x, y, bHorz, delta, xlimit, ylimit):
    if bHorz:
        ret = ( ( x+delta/2<=xlimit and  y+delta/2 <= ylimit) 
            +( x+delta <=xlimit and y+delta <= ylimit)
            +( x+delta/2 <=xlimit and y-delta/2 >= 0 )
            +( x <= xlimit and y-delta >=0 ) )
    else:
        ret = ( (x-delta/2 >=0 and y+delta/2 <= ylimit) 
            +(x-delta >=0 and y+delta <= ylimit)
            +(x+delta/2 <= xlimit and y+delta/2 <= ylimit)
            +(x+delta <= xlimit and y <= ylimit) )
    return ret

def brute_nm(n, m):
    total = 0
    for x in range(n+1):
        for y in range(m+1):
            for delta in range(1, n-x+1):
                total += tri(x,y, True,  delta,n, m)
            for delta in range(1, m-y+1):
                total += tri(x,y, False, delta,n, m)
    return total

for i in range(1,4):print(brute_nm(i,i))


8
44
124


对n * n的矩阵
看单个方向，向上生长的三角形：长度为i的水平线段，每条横线上n+1-i段，附带的两个三角形都在大矩形内的，有n+1-i条横线，附带的两个三角形只有小者在大矩形内的有 (n+1-i)//2条。  对 2*i^2 + i*(n+1-i)//2 从1到n求和。
总数 = 单个方向数再乘4

In [27]:

def algo_n0(n):
    return (sum(i*i for i in range(1,n+1)) * 2 + sum(i * ((n+1-i)//2) for i in range(1,n+1)))*4

# 对algo_n0，转换成通项公式，其中 n%2也可以转换成(-1)^n的相关形态
def algo_n(n):
    return (6*n**3 + 9*n**2 + 2*n - n%2)/2 

print(algo_n0(100), algo_n(100))

3045100 3045100.0


上面的计算，可以推出两点：结果是n的三次多项式，存在(-1)^n的部分。 使用sympy再推导一下，与手算结果比对。

In [29]:
from sympy import symbols, solve, init_printing
init_printing()

def expr_n():
    ps = symbols('a,b,c,d,e')
    n = symbols('n')
    func = ps[0] *n**3 + ps[1] *n**2 + ps[2]*n + ps[3] + ps[4]*(-1)**n
    eqs = []
    for i in range(1, 9, 1):
        eqs.append(func.subs({'n':i}) - brute_nm(i,i))
    coefs = solve(eqs, ps)
    print(coefs)
    realfunc = func.subs(coefs)
    display(realfunc)

expr_n()


{e: 1/4, d: -1/4, c: 1, b: 9/2, a: 3}


根据上面的经验，用sympy推导nm的表达式。 分为m<=n<2m 与 n>=2m，两个条件。
（开始使用统一方程组来推导，无法求解，则逐步划分条件）

In [36]:
def expr_nm():
    n,m = symbols('n m')
    vs = (n*n*n , n*n*m , n*m*m , m*m*m
            , n*n , n*m , m*m, n , m, 1
            , (-1)**m , (-1)**n)
    ps = symbols(['c%d'%i for i in range(len(vs))])
    func = sum(p*v for p,v in zip(ps, vs))

    eqs = []
    try:
        for y in range(1, 100):
            for x in range(y, 100):
                if x < 2*y:
                    eqs.append(func.subs({'n':x, 'm':y}) - brute_nm(x, y))
                if len(eqs) >= 2*len(ps):  #方程数要适当多于多项式参数
                    raise StopIteration
    except StopIteration:
        pass
    coefs_less2 = solve(eqs, ps)
    print(coefs_less2)
    realfunc_less2 = func.subs(coefs_less2)
    #print(f"m<=n<2m: ({realfunc_less2*24})/24")
    display(realfunc_less2*24) #放大24倍，方便通分展示

    eqs = []
    try:
        for x in range(1, 100):
            for y in range(1, x+1):
                if 2*y <= x:
                    eqs.append(func.subs({'n':x, 'm':y}) - brute_nm(x, y))
                    if len(eqs) >= 3*len(ps):
                        raise StopIteration
                else:
                    break
    except StopIteration:
        pass
    coefs_more2 = solve(eqs, ps)
    print(coefs_more2)
    realfunc_more2 = func.subs(coefs_more2)
    #print(f"n>=2m: ({realfunc_more2*24})/24")
    display(realfunc_more2*24) #放大24倍，方便通分展示

expr_nm()

{c11: 1/8, c10: 1/8, c6: 1/4, c8: 5/6, c3: -5/6, c9: -1/4, c5: 4, c1: 1, c7: 1/6, c2: 3, c4: 1/4, c0: -1/6}


{c6: -3/4, c10: 1/8, c9: -1/8, c7: 0, c8: 7/6, c11: 0, c5: 5, c3: -13/6, c4: 0, c2: 5, c1: 0, c0: 0}


根据上面的计算，转换成算法algo_nm如下

In [40]:

def algo_nm(n,m):
    '''  '''
    if n<m:
        n,m = m,n
    if n < 2*m:
        return (- 20*m**3 + 72*m**2*n + 6*m**2 + 24*m*n**2 
            + 96*m*n + 20*m - 4*n**3 + 6*n**2 + 4*n - 6 + 3*(-1)**m + 3*(-1)**n)/24
    else:
        return (-52*m**3 + 120*m**2*n - 18*m**2 + 120*m*n + 28*m - 
                3*(1-(-1)**m))/24

def test():
    for x in range(1,30):
        for y in range(1,x+1):
            assert(brute_nm(x,y) == algo_nm(x,y))
    print('Done')

test()

Done
